In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings = warnings.filterwarnings('ignore')

df=pd.read_csv('stock_details_5_years.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29 00:00:00-05:00,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29 00:00:00-05:00,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29 00:00:00-05:00,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29 00:00:00-05:00,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29 00:00:00-05:00,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA


In [15]:
df_allstocks = df.pivot(index='Date', columns='Company', values='Close')
df_allstocks.reset_index(inplace=True)
df_allstocks.columns.name = None
df_allstocks = df_allstocks.set_index('Date')

print(df_allstocks.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 2018-11-29 00:00:00-05:00 to 2023-11-29 00:00:00-05:00
Columns: 491 entries, A to ZTS
dtypes: float64(491)
memory usage: 4.7+ MB
None


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602962 entries, 0 to 602961
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          602962 non-null  object 
 1   Open          602962 non-null  float64
 2   High          602962 non-null  float64
 3   Low           602962 non-null  float64
 4   Close         602962 non-null  float64
 5   Volume        602962 non-null  int64  
 6   Dividends     602962 non-null  float64
 7   Stock Splits  602962 non-null  float64
 8   Company       602962 non-null  object 
dtypes: float64(6), int64(1), object(2)
memory usage: 41.4+ MB


In [10]:
# Check for missing values
print(df.isnull().sum())

Date            0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
Company         0
dtype: int64


In [13]:
# Lets download some macroeconomic data using FRED API

import pandas_datareader.data as web
import datetime

start = datetime.datetime(2018, 11, 29)
end = datetime.datetime(2023, 11, 29)

# Download the data
gdp = web.DataReader('GDP', 'fred', start, end)
unemployment = web.DataReader('UNRATE', 'fred', start, end)
cpi = web.DataReader('CPIAUCNS', 'fred', start, end)
consumer_confidence = web.DataReader('UMCSENT', 'fred', start, end)
m2 = web.DataReader('M2', 'fred', start, end)
three_month_yield = web.DataReader('TB3MS', 'fred', start, end)
ten_year_yield = web.DataReader('GS10', 'fred', start, end)
thirty_year_yield = web.DataReader('GS30', 'fred', start, end)
fed_funds_rate = web.DataReader('FEDFUNDS', 'fred', start, end)
usd_index = web.DataReader('DTWEXM', 'fred', start, end)
oil_price = web.DataReader('DCOILWTICO', 'fred', start, end)

# Merge the data
macro_data = pd.concat([gdp, unemployment, cpi, consumer_confidence, m2, three_month_yield, ten_year_yield, thirty_year_yield, fed_funds_rate, usd_index, oil_price], axis=1)

# Rename the columns

macro_data.columns = ['GDP', 'Unemployment Rate', 'CPI', 'Consumer Confidence', 'M2', '3-Month Yield', '10-Year Yield', '30-Year Yield', 'Fed Funds Rate', 'USD Index', 'Oil Price']

macro_data.tail()


,GDP,Unemployment Rate,CPI,Consumer Confidence,M2,3-Month Yield,10-Year Yield,30-Year Yield,Fed Funds Rate,USD Index,Oil Price
DATE,,,,,,,,,,,
2023-11-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.83
2023-11-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.46
2023-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.09
2023-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.56


In [14]:
# Collecting SPY ticker as well to use as dependent variable

spy = web.DataReader('SPY', 'yahoo', start, end)
spy = spy['Close']

# Merge the data
macro_data = pd.concat([macro_data, spy], axis=1)

TypeError: string indices must be integers, not 'str'